# 🧬 Simulación y Análisis de Alanina Dipéptido

## Introducción

La **alanina dipéptido** (Ace-Ala-Nme) es uno de los sistemas más simples para aprender dinámicas moleculares. Es lo suficientemente pequeño para simulaciones rápidas, pero lo suficientemente complejo para mostrar comportamientos interesantes.

### ¿Por qué Alanina Dipéptido?
- ✅ Sistema pequeño (~22 átomos)
- ✅ Simula rápidamente
- ✅ Muestra conformaciones importantes (α-hélice, β-sheet)
- ✅ Ideal para aprender análisis conformacional

### Objetivos de este Notebook:
1. Descargar y preparar la estructura
2. Configurar y ejecutar una simulación MD con OpenMM
3. Analizar la trayectoria (RMSD, ángulos diedros)
4. Crear diagrama de Ramachandran
5. Realizar análisis de componentes principales (PCA)
6. Visualizar resultados

---

## 📦 Paso 1: Importar Librerías

Primero importamos todas las librerías necesarias para la simulación y análisis.

In [ ]:
# Librerías para simulación molecular
import openmm as mm                    # Motor de simulación OpenMM
from openmm import app, unit           # Herramientas de aplicación y unidades

# Librerías para análisis de trayectorias
import mdtraj as md                    # Análisis de trayectorias MD
import MDAnalysis as mda               # Alternativa para análisis
from MDAnalysis.analysis import rms, align

# Librerías científicas
import numpy as np                     # Operaciones numéricas
import pandas as pd                    # Manejo de datos tabulares
from scipy import stats                # Estadística
from sklearn.decomposition import PCA  # Análisis de componentes principales

# Librerías para visualización
import matplotlib.pyplot as plt        # Gráficos 2D
import seaborn as sns                  # Gráficos estadísticos elegantes
import nglview as nv                   # Visualización 3D de moléculas

# Configuración de estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Utilidades
import urllib.request                  # Descargar archivos de internet
import warnings
warnings.filterwarnings('ignore')      # Suprimir advertencias menores

print("✅ Todas las librerías importadas correctamente")
print(f"   OpenMM versión: {mm.__version__}")
print(f"   MDTraj versión: {md.__version__}")

## 📥 Paso 2: Descargar la Estructura

Descargamos la estructura de alanina dipéptido desde el Protein Data Bank (PDB).

**¿Qué es un archivo PDB?**
- Es un formato de texto que contiene coordenadas 3D de átomos
- Incluye información sobre la topología molecular
- Es el estándar internacional para estructuras biomoleculares

In [ ]:
def descargar_pdb(pdb_id="1ALA", output_file="alanine_dipeptide.pdb"):
    """
    Descarga una estructura PDB desde RCSB.
    
    La función:
    1. Construye la URL del PDB
    2. Descarga el archivo
    3. Lo guarda localmente
    """
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    print(f"📥 Descargando {pdb_id} desde RCSB PDB...")
    
    try:
        urllib.request.urlretrieve(url, output_file)
        print(f"✅ Estructura guardada en: {output_file}")
        return output_file
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Descargar la estructura
pdb_file = descargar_pdb("1ALA", "alanine_dipeptide.pdb")

## 🔍 Paso 3: Explorar la Estructura

Antes de simular, veamos qué contiene nuestra estructura.

In [ ]:
# Cargar con MDTraj para análisis inicial
traj_inicial = md.load(pdb_file)

print("📊 Información de la estructura:")
print(f"   Número de átomos: {traj_inicial.n_atoms}")
print(f"   Número de residuos: {traj_inicial.n_residues}")
print(f"   Residuos: {[r.name for r in traj_inicial.topology.residues]}")
print(f"\n   Topología:")
print(traj_inicial.topology)

## 🎨 Visualización Inicial de la Estructura

Visualicemos la estructura 3D antes de simular.

In [ ]:
# Crear visualización interactiva con NGLView
view = nv.show_mdtraj(traj_inicial)

# Personalizar la visualización
view.clear_representations()  # Limpiar visualización por defecto
view.add_representation('ball+stick', selection='all')  # Modelo de bolas y palitos
view.add_representation('cartoon', selection='protein', color='cyan')  # Cartoon para proteína

# Mostrar
view

## ⚙️ Paso 4: Configurar el Sistema de Simulación

Ahora configuramos OpenMM para realizar la simulación.

### Componentes de una Simulación:
1. **Topología**: Conectividad de átomos
2. **Campo de Fuerza**: Parámetros de energía
3. **Integrador**: Algoritmo para evolucionar el sistema
4. **Sistema**: Condiciones de simulación

In [ ]:
# 1. Cargar estructura con OpenMM
pdb = app.PDBFile(pdb_file)
print("✅ Estructura cargada")

# 2. Seleccionar campo de fuerza
# AMBER14 es un campo de fuerza popular para proteínas
# Incluye parámetros para todos los aminoácidos
forcefield = app.ForceField('amber14-all.xml', 'implicit/gbn2.xml')
print("✅ Campo de fuerza AMBER14 cargado")

# 3. Crear el sistema molecular
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.NoCutoff,      # Sin corte de interacciones (sistema pequeño)
    constraints=app.HBonds,             # Restringir enlaces con hidrógeno
    rigidWater=True,                    # Agua rígida (más eficiente)
    implicitSolvent=app.GBn2            # Solvente implícito (sin agua explícita)
)
print("✅ Sistema molecular creado")

# 4. Configurar integrador de Langevin
# El integrador de Langevin simula un baño térmico (ensemble NVT)
temperature = 300 * unit.kelvin         # Temperatura: 300 K (temperatura ambiente)
friction = 1.0 / unit.picosecond        # Coeficiente de fricción
timestep = 2.0 * unit.femtosecond       # Paso de tiempo: 2 fs

integrator = mm.LangevinIntegrator(temperature, friction, timestep)
print(f"✅ Integrador configurado (T={temperature}, dt={timestep})")

# 5. Crear objeto de simulación
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
print("✅ Simulación inicializada")

## ⚡ Paso 5: Minimización de Energía

Antes de simular, minimizamos la energía para eliminar contactos malos (átomos muy cercanos).

**¿Por qué minimizar?**
- Las estructuras experimentales pueden tener átomos superpuestos
- Evita fuerzas extremas que desestabilizarían la simulación
- Encuentra la conformación de mínima energía local

In [ ]:
# Obtener energía inicial
state = simulation.context.getState(getEnergy=True)
energia_inicial = state.getPotentialEnergy()
print(f"⚡ Energía inicial: {energia_inicial}")

# Minimizar energía
print("\n🔧 Minimizando energía...")
simulation.minimizeEnergy(maxIterations=500)

# Obtener energía final
state = simulation.context.getState(getEnergy=True)
energia_final = state.getPotentialEnergy()
print(f"✅ Energía final: {energia_final}")
print(f"📉 Cambio de energía: {energia_final - energia_inicial}")

## 🌡️ Paso 6: Equilibración del Sistema

Equilibramos el sistema para que alcance la temperatura deseada.

**¿Qué es la equilibración?**
- Permite que el sistema se adapte a las condiciones de simulación
- La temperatura y energía se estabilizan
- Típicamente 0.1-1 ns dependiendo del sistema

In [ ]:
# Equilibración: 2000 pasos = 4 ps (2 fs/paso * 2000)
print("🌡️  Equilibrando sistema...")
equilibration_steps = 2000
simulation.step(equilibration_steps)
print(f"✅ Equilibración completada ({equilibration_steps} pasos = {equilibration_steps * 2} fs)")

## 🚀 Paso 7: Simulación de Producción

¡Ahora sí! Ejecutamos la simulación principal y guardamos la trayectoria.

**Parámetros de producción:**
- Duración: 20 ps (10,000 pasos × 2 fs)
- Guardamos cada 100 pasos = cada 0.2 ps
- Total de frames: 100

In [ ]:
# Configurar nombres de archivos de salida
output_dcd = "alanine_trajectory.dcd"  # Archivo de trayectoria (formato binario)
output_pdb = "alanine_final.pdb"        # Estructura final

# Configurar reporteros (guardan información durante la simulación)
# Reportero 1: Guardar trayectoria en formato DCD
simulation.reporters.append(
    app.DCDReporter(output_dcd, 100)  # Guardar cada 100 pasos
)

# Reportero 2: Imprimir estadísticas en pantalla
simulation.reporters.append(
    app.StateDataReporter(
        'simulation_log.txt',
        100,                              # Reportar cada 100 pasos
        step=True,                        # Número de paso
        time=True,                        # Tiempo de simulación
        potentialEnergy=True,             # Energía potencial
        kineticEnergy=True,               # Energía cinética
        totalEnergy=True,                 # Energía total
        temperature=True,                 # Temperatura
        speed=True                        # Velocidad de simulación
    )
)

print("🚀 Iniciando simulación de producción...")
print("   Esto puede tomar 1-2 minutos...")

# Ejecutar simulación
production_steps = 10000
simulation.step(production_steps)

# Guardar estructura final
positions = simulation.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(simulation.topology, positions, open(output_pdb, 'w'))

print("\n✅ Simulación completada exitosamente!")
print(f"   Trayectoria guardada en: {output_dcd}")
print(f"   Estructura final en: {output_pdb}")
print(f"   Tiempo total simulado: {production_steps * 2} fs = {production_steps * 2 / 1000} ps")

---

# 📊 ANÁLISIS DE LA TRAYECTORIA

Ahora que tenemos la trayectoria, ¡analicémosla!

---

## 📈 Paso 8: Cargar y Analizar Energías

Primero veamos cómo evolucionó la energía durante la simulación.

In [ ]:
# Leer el archivo de log
data = pd.read_csv('simulation_log.txt', sep=',', skiprows=0)

# Limpiar nombres de columnas (quitar espacios)
data.columns = data.columns.str.strip()

print("📊 Datos de simulación:")
print(data.head())
print(f"\n   Total de frames: {len(data)}")

In [ ]:
# Graficar energías a lo largo del tiempo
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Energía Potencial
axes[0, 0].plot(data['Time (ps)'], data['Potential Energy (kJ/mole)'], 
                color='blue', linewidth=1.5)
axes[0, 0].set_xlabel('Tiempo (ps)', fontsize=12)
axes[0, 0].set_ylabel('Energía Potencial (kJ/mol)', fontsize=12)
axes[0, 0].set_title('Energía Potencial vs Tiempo', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Energía Cinética
axes[0, 1].plot(data['Time (ps)'], data['Kinetic Energy (kJ/mole)'], 
                color='red', linewidth=1.5)
axes[0, 1].set_xlabel('Tiempo (ps)', fontsize=12)
axes[0, 1].set_ylabel('Energía Cinética (kJ/mol)', fontsize=12)
axes[0, 1].set_title('Energía Cinética vs Tiempo', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Energía Total
axes[1, 0].plot(data['Time (ps)'], data['Total Energy (kJ/mole)'], 
                color='green', linewidth=1.5)
axes[1, 0].set_xlabel('Tiempo (ps)', fontsize=12)
axes[1, 0].set_ylabel('Energía Total (kJ/mol)', fontsize=12)
axes[1, 0].set_title('Energía Total vs Tiempo', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Temperatura
axes[1, 1].plot(data['Time (ps)'], data['Temperature (K)'], 
                color='orange', linewidth=1.5)
axes[1, 1].axhline(y=300, color='red', linestyle='--', label='Target (300 K)')
axes[1, 1].set_xlabel('Tiempo (ps)', fontsize=12)
axes[1, 1].set_ylabel('Temperatura (K)', fontsize=12)
axes[1, 1].set_title('Temperatura vs Tiempo', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('energia_analisis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Estadísticas de Energía:")
print(f"   Energía Potencial promedio: {data['Potential Energy (kJ/mole)'].mean():.2f} ± {data['Potential Energy (kJ/mole)'].std():.2f} kJ/mol")
print(f"   Temperatura promedio: {data['Temperature (K)'].mean():.2f} ± {data['Temperature (K)'].std():.2f} K")

## 🎬 Paso 9: Visualizar la Trayectoria

Veamos cómo se movió la molécula durante la simulación.

In [ ]:
# Cargar trayectoria con MDTraj
traj = md.load(output_dcd, top=pdb_file)

print(f"📊 Información de la trayectoria:")
print(f"   Número de frames: {traj.n_frames}")
print(f"   Tiempo total: {traj.n_frames * 0.2} ps")
print(f"   Número de átomos: {traj.n_atoms}")

# Visualizar trayectoria completa
view_traj = nv.show_mdtraj(traj)
view_traj.clear_representations()
view_traj.add_representation('cartoon', selection='protein', color='blue')
view_traj.add_representation('licorice', selection='protein')
view_traj

## 📏 Paso 10: Análisis RMSD (Root Mean Square Deviation)

El RMSD mide cuánto se desvía la estructura de la posición inicial.

**Interpretación:**
- RMSD bajo (< 1 Å): Estructura estable
- RMSD alto (> 3 Å): Cambio conformacional significativo
- RMSD creciente: Sistema explorando nuevas conformaciones

In [ ]:
# Calcular RMSD respecto al primer frame
# Alineamos primero para remover traslaciones y rotaciones
traj_aligned = traj.superpose(traj, frame=0)

# Calcular RMSD de todos los átomos
rmsd_all = md.rmsd(traj_aligned, traj_aligned, frame=0) * 10  # Convertir nm a Å

# Calcular RMSD solo del backbone (cadena principal)
backbone_atoms = traj.topology.select('backbone')
rmsd_backbone = md.rmsd(traj_aligned, traj_aligned, frame=0, atom_indices=backbone_atoms) * 10

# Graficar RMSD
fig, ax = plt.subplots(figsize=(12, 6))

time = np.arange(len(rmsd_all)) * 0.2  # Tiempo en ps

ax.plot(time, rmsd_all, label='Todos los átomos', linewidth=2, alpha=0.7)
ax.plot(time, rmsd_backbone, label='Backbone', linewidth=2, alpha=0.7)

ax.set_xlabel('Tiempo (ps)', fontsize=14)
ax.set_ylabel('RMSD (Å)', fontsize=14)
ax.set_title('RMSD vs Tiempo - Alanina Dipéptido', fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('rmsd_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Estadísticas RMSD:")
print(f"   RMSD promedio (todos): {rmsd_all.mean():.3f} ± {rmsd_all.std():.3f} Å")
print(f"   RMSD promedio (backbone): {rmsd_backbone.mean():.3f} ± {rmsd_backbone.std():.3f} Å")
print(f"   RMSD máximo: {rmsd_all.max():.3f} Å")

## 🔄 Paso 11: Diagrama de Ramachandran

El diagrama de Ramachandran muestra los ángulos diedros φ (phi) y ψ (psi) del backbone.

**¿Qué nos dice?**
- **Región α-helix**: φ ≈ -60°, ψ ≈ -45°
- **Región β-sheet**: φ ≈ -120°, ψ ≈ +120°
- **Región PPII**: φ ≈ -60°, ψ ≈ +150°

Para alanina dipéptido, esperamos ver transiciones entre estas regiones.

In [ ]:
# Calcular ángulos diedros phi y psi
phi_angles = md.compute_phi(traj)[1]  # [1] porque devuelve (indices, angulos)
psi_angles = md.compute_psi(traj)[1]

# Convertir de radianes a grados
phi_degrees = np.degrees(phi_angles).flatten()
psi_degrees = np.degrees(psi_angles).flatten()

print(f"📐 Ángulos calculados:")
print(f"   Phi: {len(phi_degrees)} valores")
print(f"   Psi: {len(psi_degrees)} valores")

# Crear diagrama de Ramachandran
fig, ax = plt.subplots(figsize=(10, 10))

# Scatter plot de phi vs psi
scatter = ax.scatter(phi_degrees, psi_degrees, 
                     c=time[:len(phi_degrees)],  # Color por tiempo
                     cmap='viridis', 
                     alpha=0.6, 
                     s=50,
                     edgecolors='black',
                     linewidth=0.5)

# Añadir colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Tiempo (ps)', fontsize=12)

# Marcar regiones características
# Región α-helix
ax.plot(-60, -45, 'r*', markersize=20, label='α-helix')
# Región β-sheet
ax.plot(-120, 120, 'b*', markersize=20, label='β-sheet')
# Región PPII
ax.plot(-60, 150, 'g*', markersize=20, label='PPII')

# Configurar ejes
ax.set_xlabel('φ (Phi) - grados', fontsize=14)
ax.set_ylabel('ψ (Psi) - grados', fontsize=14)
ax.set_title('Diagrama de Ramachandran - Alanina Dipéptido', 
             fontsize=16, fontweight='bold')
ax.set_xlim(-180, 180)
ax.set_ylim(-180, 180)
ax.axhline(0, color='gray', linestyle='--', alpha=0.3)
ax.axvline(0, color='gray', linestyle='--', alpha=0.3)
ax.legend(fontsize=12, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ramachandran_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Estadísticas de ángulos diedros:")
print(f"   Phi promedio: {phi_degrees.mean():.2f}° ± {phi_degrees.std():.2f}°")
print(f"   Psi promedio: {psi_degrees.mean():.2f}° ± {psi_degrees.std():.2f}°")

## 📊 Paso 12: Análisis de Componentes Principales (PCA)

PCA nos ayuda a identificar los movimientos más importantes de la molécula.

**¿Qué es PCA?**
- Reduce la dimensionalidad de los datos
- Identifica las direcciones de mayor varianza
- PC1 = movimiento más importante
- PC2 = segundo movimiento más importante

In [ ]:
# Preparar datos para PCA
# Usamos solo el backbone para simplificar
backbone_xyz = traj_aligned.xyz[:, backbone_atoms, :]

# Reshape: (n_frames, n_atoms * 3)
n_frames, n_atoms, _ = backbone_xyz.shape
data_pca = backbone_xyz.reshape(n_frames, -1)

print(f"🔍 Preparando PCA:")
print(f"   Shape de datos: {data_pca.shape}")
print(f"   ({n_frames} frames, {n_atoms} átomos × 3 coordenadas)")

# Aplicar PCA
pca = PCA(n_components=5)  # Calculamos 5 componentes principales
pc_coords = pca.fit_transform(data_pca)

# Varianza explicada
variance_ratio = pca.explained_variance_ratio_

print(f"\n📊 Varianza explicada por cada PC:")
for i, var in enumerate(variance_ratio, 1):
    print(f"   PC{i}: {var*100:.2f}%")
print(f"\n   Total (PC1-PC5): {variance_ratio.sum()*100:.2f}%")

In [ ]:
# Visualizar PCA
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Varianza explicada
axes[0].bar(range(1, 6), variance_ratio * 100, alpha=0.7, color='steelblue')
axes[0].plot(range(1, 6), np.cumsum(variance_ratio * 100), 
             'ro-', linewidth=2, markersize=8, label='Acumulada')
axes[0].set_xlabel('Componente Principal', fontsize=12)
axes[0].set_ylabel('Varianza Explicada (%)', fontsize=12)
axes[0].set_title('Varianza Explicada por PCA', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Proyección en PC1 vs PC2
scatter = axes[1].scatter(pc_coords[:, 0], pc_coords[:, 1], 
                         c=time[:len(pc_coords)], 
                         cmap='plasma', 
                         alpha=0.7,
                         s=60,
                         edgecolors='black',
                         linewidth=0.5)

cbar = plt.colorbar(scatter, ax=axes[1])
cbar.set_label('Tiempo (ps)', fontsize=12)

axes[1].set_xlabel(f'PC1 ({variance_ratio[0]*100:.1f}% varianza)', fontsize=12)
axes[1].set_ylabel(f'PC2 ({variance_ratio[1]*100:.1f}% varianza)', fontsize=12)
axes[1].set_title('Proyección en Espacio de Componentes Principales', 
                  fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pca_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 🎨 Paso 13: Visualizar Movimiento a lo largo de PC1

Veamos cómo se mueve la molécula a lo largo del primer componente principal.

In [ ]:
# Encontrar los extremos de PC1
pc1_min_idx = np.argmin(pc_coords[:, 0])
pc1_max_idx = np.argmax(pc_coords[:, 0])

print(f"🎬 Extremos de PC1:")
print(f"   Mínimo: frame {pc1_min_idx} (tiempo {pc1_min_idx * 0.2:.1f} ps)")
print(f"   Máximo: frame {pc1_max_idx} (tiempo {pc1_max_idx * 0.2:.1f} ps)")

# Crear trayectoria con solo estos extremos
extreme_frames = traj[[pc1_min_idx, pc1_max_idx]]

# Visualizar
view_pc1 = nv.show_mdtraj(extreme_frames)
view_pc1.clear_representations()
view_pc1.add_representation('cartoon', selection='protein', color='cyan')
view_pc1.add_representation('licorice', selection='protein')
view_pc1

## 📋 Paso 14: Resumen y Conclusiones

Generemos un reporte final con todas las métricas.

In [ ]:
# Crear reporte final
print("="*70)
print("📊 REPORTE FINAL - SIMULACIÓN DE ALANINA DIPÉPTIDO")
print("="*70)

print("\n🔧 PARÁMETROS DE SIMULACIÓN:")
print(f"   Campo de fuerza: AMBER14")
print(f"   Solvente: Implícito (GBn2)")
print(f"   Temperatura: 300 K")
print(f"   Timestep: 2 fs")
print(f"   Duración: {production_steps * 2 / 1000} ps")
print(f"   Frames guardados: {traj.n_frames}")

print("\n⚡ ENERGÍAS:")
print(f"   Energía potencial promedio: {data['Potential Energy (kJ/mole)'].mean():.2f} ± {data['Potential Energy (kJ/mole)'].std():.2f} kJ/mol")
print(f"   Temperatura promedio: {data['Temperature (K)'].mean():.2f} ± {data['Temperature (K)'].std():.2f} K")

print("\n📏 ESTABILIDAD ESTRUCTURAL:")
print(f"   RMSD promedio (todos): {rmsd_all.mean():.3f} ± {rmsd_all.std():.3f} Å")
print(f"   RMSD promedio (backbone): {rmsd_backbone.mean():.3f} ± {rmsd_backbone.std():.3f} Å")
print(f"   RMSD máximo: {rmsd_all.max():.3f} Å")

print("\n📐 ÁNGULOS DIEDROS:")
print(f"   Phi (φ) promedio: {phi_degrees.mean():.2f}° ± {phi_degrees.std():.2f}°")
print(f"   Psi (ψ) promedio: {psi_degrees.mean():.2f}° ± {psi_degrees.std():.2f}°")

print("\n🔍 ANÁLISIS PCA:")
print(f"   PC1 explica: {variance_ratio[0]*100:.2f}% de la varianza")
print(f"   PC2 explica: {variance_ratio[1]*100:.2f}% de la varianza")
print(f"   PC1+PC2 explican: {(variance_ratio[0]+variance_ratio[1])*100:.2f}% de la varianza total")

print("\n📁 ARCHIVOS GENERADOS:")
print(f"   ✅ {output_dcd} - Trayectoria")
print(f"   ✅ {output_pdb} - Estructura final")
print(f"   ✅ simulation_log.txt - Log de simulación")
print(f"   ✅ energia_analisis.png - Gráficos de energía")
print(f"   ✅ rmsd_analysis.png - Análisis RMSD")
print(f"   ✅ ramachandran_plot.png - Diagrama de Ramachandran")
print(f"   ✅ pca_analysis.png - Análisis PCA")

print("\n" + "="*70)
print("🎉 ¡Análisis completado exitosamente!")
print("="*70)

## 🎓 Conclusiones y Aprendizajes

### ¿Qué aprendimos?

1. **Configuración de simulaciones**: Cómo preparar un sistema molecular
2. **Minimización y equilibración**: Por qué son necesarias
3. **Análisis de energías**: Cómo verificar estabilidad
4. **RMSD**: Medir desviaciones estructurales
5. **Ramachandran**: Analizar conformaciones del backbone
6. **PCA**: Identificar movimientos colectivos

### Próximos pasos:

- Aumentar el tiempo de simulación (100 ps - 1 ns)
- Simular en agua explícita
- Analizar clustering de conformaciones
- Calcular free energy landscapes
- Probar con sistemas más grandes (proteínas)

### Recursos adicionales:

- [OpenMM Documentation](http://docs.openmm.org/)
- [MDTraj Tutorials](http://mdtraj.org/)
- [Best Practices in MD](https://www.mdanalysis.org/)

---

**¡Felicidades!** Has completado tu primera simulación de dinámica molecular 🎉